# Kernel SVM

In [1]:
# Para tratamento dos dados
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt


## Importing the dataset

In [2]:
dataset = pd.read_csv('Data.csv')
dataset.head()

,sentimento,empresa,tempo_reclamacao,status,titulo,seguro,sinistro,prestamista,carro,cancelamento,protecao,atendimento,rembolso
0,Negativo,HDI Seguros,4,Não resolvido,Demora no reparo do veículo,False,False,False,False,False,False,False,False
1,Negativo,HDI Seguros,13,Não resolvido,Reembolso honorários,False,False,False,False,False,False,False,True
2,Negativo,HDI Seguros,15,Não resolvido,Carro reserva sinistro 010425351170591,False,True,False,True,False,False,False,False
3,Negativo,HDI Seguros,15,Não resolvido,Terceirizado quebrou uma peça do meu veículo,False,False,False,False,False,False,False,False
4,Negativo,HDI Seguros,16,Resolvido,DEMORA NA ENTREGA DAS PEÇAS,False,False,False,False,False,False,False,False


In [3]:
dataset.drop(columns=["titulo","empresa"],inplace=True)

In [4]:
dataset.head()

,sentimento,tempo_reclamacao,status,seguro,sinistro,prestamista,carro,cancelamento,protecao,atendimento,rembolso
0,Negativo,4,Não resolvido,False,False,False,False,False,False,False,False
1,Negativo,13,Não resolvido,False,False,False,False,False,False,False,True
2,Negativo,15,Não resolvido,False,True,False,True,False,False,False,False
3,Negativo,15,Não resolvido,False,False,False,False,False,False,False,False
4,Negativo,16,Resolvido,False,False,False,False,False,False,False,False


In [5]:
# Dividindo Variáveis
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

In [6]:
print(y)

['Negativo' 'Negativo' 'Negativo' ... 'Positivo' 'Positivo' 'Positivo']


In [7]:
print(X)

[[4 'Não resolvido' False ... False False False]
 [13 'Não resolvido' False ... False False True]
 [15 'Não resolvido' False ... False False False]
 ...
 [22 'Resolvido' False ... False False False]
 [51 'Resolvido' False ... False False False]
 [6 'Não resolvido' False ... False False False]]


## Encoding categorical data

### Encoding the Independent Variable - Transformando varivaies independentes em dummy

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2,3,4,5,6,7,8,9])], remainder='passthrough') # trandforme tem 3 argumentos: método, transformação, coluna
X = np.array(ct.fit_transform(X)) # transformando variáveis

In [9]:
X

array([[1.0, 0.0, 1.0, ..., 1.0, 0.0, 4],
       [1.0, 0.0, 1.0, ..., 0.0, 1.0, 13],
       [1.0, 0.0, 1.0, ..., 1.0, 0.0, 15],
       ...,
       [0.0, 1.0, 1.0, ..., 1.0, 0.0, 22],
       [0.0, 1.0, 1.0, ..., 1.0, 0.0, 51],
       [1.0, 0.0, 1.0, ..., 1.0, 0.0, 6]], dtype=object)

### Encoding the Dependent Variable -  Transformação de variável dependente

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # transformação dummy para duas categorias
y = le.fit_transform(y) # transformando

In [11]:
y

array([0, 0, 0, ..., 1, 1, 1])

## Splitting the dataset into the Training set and Test set

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling - Padronização de Valores

Utilizamos a padronização de valores quando temos valores muitos descrepetantes. Desta forma, os valores são transformados em um intervalo de -3 a 3.

*Obs*: caso os valores já estiveram no intervalo de -3 a 3 não será necessário padronizalos.

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

## Training the Kernel SVM model on the Training set

In [14]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

## Making the Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[198  67]
 [ 12  78]]


0.7774647887323943